In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from os import getenv
from util_functions import get_examples_for_prompt, read_clinical_docs, validate_relations, write_pubtator_data
from tqdm.auto import tqdm
import sys
sys.path.append("../.")
from config import Config
from prompt_config import Prompt_Config

load_dotenv()

/Users/g.karunakaran/Non-work/clinical-relation-extraction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = OpenAI(
  # base_url="https://openrouter.ai/api/v1",
  # api_key=getenv("OPENROUTER_API_KEY"),
  api_key=getenv("OPENAI_API_KEY"),
)

In [3]:
lang = "eu"
gen_config = Config(lang)
prompt_config = Prompt_Config(lang)

In [4]:
few_shot_examples = get_examples_for_prompt(gen_config.DATASET_PATH, prompt_config.few_shot_doc_ids)

In [5]:
message_list = [
{
"role" : "system",
"content" : f"You are provided with a clinical statement in {prompt_config.language}. Your task is Relation Extraction. Extract mentions of laboratory tests and their results from the statement. Note: the result is written first in the output and then the name of the test. They are separated with a '|'. Print only the relations if any and nothing else."
},
{
"role" : "user",
"content" : few_shot_examples[0]['Text']
},
{
"role" : "assistant",
"content" : few_shot_examples[0]['Output']
},
{
"role" : "user",
"content" : few_shot_examples[1]['Text']
},
{
"role" : "assistant",
"content" : few_shot_examples[0]['Output']
},
]

In [6]:
docs = read_clinical_docs(gen_config.TEST_DATASET)

In [7]:
# model_name = "gpt-4o"
model_name = "gpt-4-turbo-2024-04-09"

In [8]:
relations = []

for doc in tqdm(docs):
    chat_prompt = [*message_list, {
        "role" : "user",
        "content" : doc[1]
        }]
    
    completion = client.chat.completions.create(
    model=model_name,
    messages=chat_prompt
  )
    
  #   completion = client.chat.completions.create(
  #   model=model_name,
  #   messages=chat_prompt,
  #   temperature=0.0
  # )
    relations.append(completion.choices[0].message.content)

  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 80/80 [03:59<00:00,  2.99s/it]


In [9]:
relations

['Sorry, but no specific laboratory tests or their results were mentioned in the statement provided.',
 '144/83 mm Hg | Tentsio arteriala\n98 tau/min | bihotz-maiztasuna\n% 99 | O2Sat\n40 dB | transmisioko hipoakusia',
 'There are no laboratory tests or results mentioned in this clinical statement related to the trauma scenario provided. The focus is on the physical incident, diagnosis, radiography, surgery, and post-operative care, without specific reference to laboratory test values or results.',
 '',
 '',
 '39 ºC | tenperatura',
 'positiboa | 13/14C-urearen arnas testa',
 '140/95 mmHg | Tentsio arteriала',
 '% 45 | eiekzioaren frakzioa',
 '235 mg/dL | kolesterol osoa\n150 mg/dL | LDL-kolesterola\n45 mg/dL | HDL-kolesterola\n140/97 mm-Hg | Tentsio arteriala',
 'No relations were mentioned regarding laboratory tests and their results in the statement provided.',
 '7 puntu | ikus-eskala analogikoa (EVA)',
 '4 kg | pisu galera',
 '',
 '% 74,5 | Oxigenoaren batez besteko saturazioa\n89 |

In [10]:
pubtator_data = validate_relations(relations, docs, gen_config.TEST_TOKEN_DATA)

GPT made an oopsie - Sorry, but no specific laboratory tests or their results were mentioned in the statement provided.
100023 (1269, 1274) (1245, 1268) failed assertion
GPT made an oopsie - There are no laboratory tests or results mentioned in this clinical statement related to the trauma scenario provided. The focus is on the physical incident, diagnosis, radiography, surgery, and post-operative care, without specific reference to laboratory test values or results.
100045 (97, 102) (106, 117) failed assertion
GPT made an oopsie - No relations were mentioned regarding laboratory tests and their results in the statement provided.
GPT made an oopsie - Ez dago laborategiko proben edo emaitzen aipamenik testuan.
GPT made an oopsie - Ez dago laborategiko proben emaitza zehatzik testu honetan adierazita.
GPT made an oopsie - No laboratory test results or names are explicitly mentioned in the given text related to their results, hence there is nothing to output based on the given instruction

In [12]:
write_pubtator_data(lang, model_name, pubtator_data, docs)